# Thief Detector
## This task tests your Image Processing skills to build a motion detection algorithm that alarms you when you have an unwanted visitor in your home.

## Steps
- 1. Get the live video feed from your webcam
- 2. Fix a scene (the place you want to monitor) and store it as a reference background image
    - Store the first frame as the reference background frame
- 3. For every frame, check if there is any unwanted object inside the scene you are monitoring
    - Use **Background Subtraction** concept (**cv2.absdiff( )**)
        - Subtract the current frame from the reference background image(frame) to see the changes in the scene
        - If there is enormous amount of pixels distrubed in the subtraction result image
            - unwanted visitor (place is unsafe --> alarm the authorities)
        - If there is no enormous amount of pixels distrubed in the subtraction result image
            - no unwanted visitor (place is safe)
- 4. Output the text **"UNSAFE"** in **red** color on the top right of the frame when there is an intruder in the scene.
- 5. Save the live feed
- 6. Submit the (.ipynb) file

In [1]:
import cv2

## Get live video feed from webcam [10 points]

In [2]:
cap = cv2.VideoCapture(0)

## Read first frame, convert to Grayscale and store it as reference background image [10 points]

In [11]:
frame = cap.read()[1]
background = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

## Compute Absolute Difference between Current and First frame [20 points]

In [13]:
frame = cap.read()[1]
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
diff = cv2.absdiff(background, gray)

## Apply threshold [5 points]

In [15]:
thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)[1]
# Dilate the thresholded image to fill in holes
thresh = cv2.dilate(thresh, None, iterations=2)

## Find contours [10 points]

In [17]:
contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

## Check if contourArea is large and draw rectangle around the object, output "UNSAFE" text in red color [30 points]

In [20]:
if len(contours) > 0:
    cv2.putText(frame, "UNSAFE", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
else:
    cv2.putText(frame, "SAFE", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

## Display images [10 points]

In [24]:
cv2.imshow("Frame with bounding box", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Release objects [5 points]

In [25]:
cap.release()
cv2.destroyAllWindows()

## Putting everything together

In [59]:
import time

cap = cv2.VideoCapture(0)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('thief_capture.mp4', fourcc, 20.0, (width, height))

frame0 = cap.read()[1]
# Give some time for the camera to adjust
time.sleep(1)
frame = cap.read()[1]
background = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

cv2.imshow("Background", background)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    diff = cv2.absdiff(background, gray)

    thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)

    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    frame_with_text = frame.copy()

    flag = False

    for contour in contours:
        area = cv2.contourArea(contour)

        if area > 30000:
            flag = True
        
    if flag:
        cv2.putText(frame_with_text, "UNSAFE", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame_with_text, (x, y), (x + w, y + h), (0, 0, 255), 2)
    else:
        cv2.putText(frame_with_text, "SAFE", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Video Feed", frame_with_text)

    out.write(frame_with_text)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()